<a target="_blank" href="https://colab.research.google.com/github/yelallioui/Python-DataScience-Master-IA-GI/blob/main/Notebooks/01_NumPy.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Ouvrir dans Colab"/>
</a>


# 🔢 NumPy — Calcul scientifique et tableaux
_Master IA-GI — Notebook 3_

Ce notebook suit et met en pratique le **support de cours NumPy** que vous avez partagé (fondamentaux, génération aléatoire, ufuncs, math/stat/algèbre, algèbre linéaire).

**Objectifs d’apprentissage**
- Comprendre la structure `ndarray` et manipuler **dimensions, shape, dtype, view/copy, reshape, slicing, indexing**  
- Utiliser les **constructeurs** (ones, zeros, arange, linspace, empty, full, eye, etc.)  
- Maîtriser les **ufuncs** et la **vectorisation**  
- Générer des **données aléatoires** et simuler des **distributions**  
- Appliquer les **fonctions mathématiques/statistiques** et l’**algèbre linéaire** (`np.linalg`)  

**Pré-requis** : Notebooks 1 & 2  
**Durée estimée** : 3h



---
## 0) ⚙️ Préparation & version


In [ ]:

import numpy as np, math, sys, platform
print("NumPy version:", np.__version__)
print("Python:", sys.version.split()[0], "| Platform:", platform.platform())



---
## 1) Fondamentaux de `ndarray` — dimensions, shape, dtype
**À retenir**
- `ndarray` = tableau **N-dimensionnel** homogène (un seul `dtype`)
- `ndim`, `shape`, `size`, `dtype`
- Indexation, slicing, **views** vs **copies**


In [ ]:

import numpy as np

# Création 0D, 1D, 2D, 3D
a0 = np.array(42)                     # 0-D
a1 = np.array([1, 2, 3, 4, 5])        # 1-D
a2 = np.array([[1,2,3],[4,5,6]])      # 2-D
a3 = np.array([[[1,2,3],[4,5,6]],
               [[7,8,9],[10,11,12]]]) # 3-D

print("ndim:", a3.ndim, "| shape:", a3.shape, "| size:", a3.size, "| dtype:", a3.dtype)



### Indexation & slicing
- Indexation standard (positive & negative)
- Slicing `[start:stop:step]`
- **Views** : la plupart des slices renvoient une **vue** (pas une copie)


In [ ]:

x = np.arange(10)      # 0..9
view = x[2:8:2]        # vue sur x
view[:] = -1           # modifie x
print("x:", x, "| view:", view)

y = x.copy()           # vraie copie
y[0] = 999
print("copy vs original:", y[0], x[0])



**Exercice 1.1 — Slicing 2D**  
Crée un `arr` de shape `(5, 6)` avec `arange` puis reshape. Extrais :
1) Les 3 premières lignes  
2) Les colonnes paires  
3) Un sous-bloc central (2x3)  

<details>
<summary>✅ Solution</summary>

```python
arr = np.arange(30).reshape(5,6)
s1 = arr[:3, :]          # 3 premières lignes
s2 = arr[:, ::2]         # colonnes paires
s3 = arr[1:3, 2:5]       # bloc central 2x3
(s1.shape, s2.shape, s3.shape), (s1, s2, s3)
```
</details>



---
## 2) Constructeurs & gestion des types (`dtype`)
**Constructeurs clés**
- `np.array`, `np.zeros`, `np.ones`, `np.empty`, `np.full`, `np.eye`
- `np.arange`, `np.linspace`
- `astype` pour convertir le `dtype`


In [ ]:

z = np.zeros((2,3), dtype=np.float32)
o = np.ones((2,3), dtype=np.int32)
f = np.full((2,3), fill_value=7.5)
I = np.eye(4)
seq = np.arange(0, 1.0, 0.2)           # pas fixe
lin = np.linspace(0, 1, num=5)         # nombre de points

print(z.dtype, o.dtype, f.dtype)
print("I:\n", I)
print("seq:", seq, "| lin:", lin)

# dtype
a = np.array([1, 2, 3], dtype=np.int8)
b = a.astype(np.float64)
a.dtype, b.dtype



**Exercice 2.1 — Grille**  
Crée une grille 2D `X` de shape `(4,5)` contenant des entiers de 10 à 29 (ordre ligne-par-ligne) puis convertis-la en `float64`.

<details>
<summary>✅ Solution</summary>

```python
X = np.arange(10, 30).reshape(4,5).astype(np.float64)
X.dtype, X.shape, X[:2]
```
</details>



---
## 3) Reshape & réarrangements
- `reshape`, dimension inconnue `-1`
- `ravel()` (vue si possible) vs `flatten()` (copie)
- `squeeze()` pour supprimer les axes de dimension 1
- `transpose`, `swapaxes`
- `stack` (`vstack`, `hstack`, `dstack`), `concatenate`, `split`


In [ ]:

A = np.arange(12).reshape(3,4)
A_T = A.T
flat_view = A.ravel()      # souvent vue
flat_copy = A.flatten()    # copie
B = np.expand_dims(np.arange(6), axis=1)  # shape (6,1)
B_sq = np.squeeze(B)       # enlève l'axe de taille 1

C = np.hstack([A, A])      # concat horizontal
D = np.vstack([A, A])      # concat vertical
A.shape, A_T.shape, flat_view.shape, flat_copy.shape, B.shape, B_sq.shape, C.shape, D.shape



**Exercice 3.1 — Split/Stack**
1) Crée `M = arange(24).reshape(4,6)`  
2) Sépare `M` en 3 blocs verticaux égaux  
3) Recolle les 3 blocs dans l’ordre `[bloc3, bloc1, bloc2]`

<details>
<summary>✅ Solution</summary>

```python
M = np.arange(24).reshape(4,6)
b1, b2, b3 = np.hsplit(M, 3)
R = np.hstack([b3, b1, b2])
(R.shape, R[:2])
```
</details>



---
## 4) Indexation avancée & **boolean masking**
- Sélection par listes/arrays d’indices
- Masques booléens (`arr[mask]`)
- `where`, `nonzero`, `argmax/argmin`, `take`, `put`


In [ ]:

rng = np.random.default_rng(42)
arr = rng.integers(0, 20, size=(5,6))
mask = (arr % 2 == 0) & (arr > 5)
filtered = arr[mask]

idx_rows = [0,2,4]
idx_cols = [1,3]
sub = arr[np.ix_(idx_rows, idx_cols)]

np.where(mask, arr, -1)[:2], filtered[:10], sub



**Exercice 4.1 — Nettoyage par masque**  
À partir d’un `arr` aléatoire de shape `(6,6)` dans `[−5, 10]`, remplace **toutes les valeurs négatives** par 0, puis :
- compte le nombre de zéros
- calcule la moyenne des valeurs > 0

<details>
<summary>✅ Solution</summary>

```python
rng = np.random.default_rng(0)
arr = rng.integers(-5, 11, size=(6,6))
arr[arr < 0] = 0
zeros = np.count_nonzero(arr == 0)
mean_pos = arr[arr > 0].mean()
zeros, mean_pos, arr
```
</details>



---
## 5) ufuncs & vectorisation
- ufuncs élémentaires : `add`, `subtract`, `multiply`, `divide`, `power`, `mod`
- ufuncs math : `sin`, `cos`, `exp`, `log`, `sqrt`, `abs`…
- Arguments `out`, `where`, `dtype`
- Broadcasting : règles d’alignement des shapes


In [ ]:

x = np.linspace(0, 2*np.pi, 6)
y = np.sin(x) + np.cos(x/2)
# broadcasting
A = np.arange(6).reshape(2,3)   # (2,3)
b = np.array([10, 100, 1000])   # (3,)
B = A + b                        # broadcast (2,3)+(3,) -> (2,3)
x, y, A, b, B



**Exercice 5.1 — Normalisation vectorisée**  
Écris une fonction `minmax(X)` qui normalise un `ndarray` **sur chaque colonne** dans `[0,1]` (sans boucles Python).

<details>
<summary>✅ Solution</summary>

```python
def minmax(X):
    X = np.asarray(X, dtype=float)
    mn = X.min(axis=0, keepdims=True)
    mx = X.max(axis=0, keepdims=True)
    return (X - mn) / (mx - mn + 1e-12)

Z = np.arange(12).reshape(4,3)
minmax(Z)
```
</details>



---
## 6) Agrégations, différences, tri, ensembles
- `sum`, `prod`, `mean`, `std`, `var`, `cumsum`, `cumprod`
- `diff` (différences discrètes)
- `sort`, `argsort`, `lexsort`, `partition`, `searchsorted`
- `unique`, `intersect1d`, `union1d`, `setdiff1d`, `setxor1d`


In [ ]:

r = np.array([3, 1, 4, 1, 5, 9, 2])
print("sum:", r.sum(), "| mean:", r.mean(), "| std:", r.std())
print("cumsum:", r.cumsum())
print("diff:", np.diff(r))

print("sorted:", np.sort(r))
print("argsort:", np.argsort(r))
print("unique:", np.unique(r))

a = np.array([1,2,3,4])
b = np.array([3,4,5,6])
print("intersect:", np.intersect1d(a,b), "| union:", np.union1d(a,b))



**Exercice 6.1 — Tri multi-clés**  
Construis deux arrays 1D `keys1` (classe) et `keys2` (note), puis trie des `noms` par **classe croissante puis note décroissante** avec `lexsort` (astuce : trier par `-keys2` pour décroissant).

<details>
<summary>✅ Solution</summary>

```python
noms = np.array(["Ali","Sara","Yasmin","Youssef","Aya"])
classe = np.array([2,1,1,2,1])
note = np.array([14.5, 16.0, 12.0, 13.0, 16.0])
# lexsort trie du dernier vers le premier
idx = np.lexsort((-note, classe))
noms[idx], classe[idx], note[idx]
```
</details>



---
## 7) Génération aléatoire & distributions (`np.random`)
- Utiliser l’API moderne `Generator` via `np.random.default_rng(seed)`
- **Distributions** : `normal`, `uniform`, `binomial`, `poisson`, `exponential`…
- **Permutation** & **shuffle**


In [ ]:

rng = np.random.default_rng(123)

# Échantillons
U = rng.uniform(low=-1.0, high=1.0, size=(2,5))
N = rng.normal(loc=0.0, scale=1.0, size=10)
B = rng.binomial(n=10, p=0.3, size=6)
P = rng.poisson(lam=4.0, size=6)

# Permutations
arr = np.arange(8)
rng.shuffle(arr)     # in-place
perm = rng.permutation(8)  # renvoie une copie permutée

U, N[:5], B, P, arr, perm



**Exercice 7.1 — Simulation**  
Simule 10 000 lancers d’un dé **biaisé** (`P(6)=0.3`, autres faces égales) et estime empiriquement la probabilité de chaque face.

<details>
<summary>✅ Solution</summary>

```python
rng = np.random.default_rng(7)
p6 = 0.3
p = np.array([ (1-p6)/5 ]*5 + [p6])  # faces 1..6
samples = rng.choice(np.arange(1,7), size=10_000, p=p)
counts = np.bincount(samples, minlength=7)[1:]  # ignore index 0
counts / counts.sum()
```
</details>



---
## 8) Fonctions mathématiques & statistiques
- Logs : `log`, `log10`, `log2`, `log1p`
- Trigo : `sin`, `cos`, `tan`, `arcsin`, `arccos`, `arctan`, conversions degrés/radians
- Hyperboliques : `sinh`, `cosh`, `tanh`
- Stat : `mean`, `median`, percentiles `quantile`, `nan*` (nanmean, nanstd, ...)


In [ ]:

x = np.array([1e-9, 1, 10, 100])
print("log:", np.log(x))
print("log10:", np.log10(x), "| log2:", np.log2(x), "| log1p:", np.log1p(x))

ang_deg = np.array([0, 30, 90])
ang_rad = np.deg2rad(ang_deg)
print("sin:", np.sin(ang_rad), "| back to deg:", np.rad2deg(ang_rad))

w = np.array([1, np.nan, 3, np.nan, 5])
np.nanmean(w), np.nanstd(w), np.quantile(np.nan_to_num(w, nan=0.0), [0.25, 0.5, 0.75])



**Exercice 8.1 — Z-score**  
Écris une fonction `zscore(x, axis=None)` qui standardise `x` : `(x-mean)/std` en ignorant les `NaN`.

<details>
<summary>✅ Solution</summary>

```python
def zscore(x, axis=None, eps=1e-12):
    x = np.asarray(x, dtype=float)
    mu = np.nanmean(x, axis=axis, keepdims=True)
    sd = np.nanstd(x, axis=axis, keepdims=True)
    return (x - mu) / (sd + eps)

X = np.array([[1,2,np.nan],[4,5,6]], dtype=float)
zscore(X, axis=0)
```
</details>



---
## 9) Algèbre linéaire (`np.linalg`)
- Transposée, diagonale, trace
- Produit matriciel : `@` ou `np.matmul`
- Déterminant `det`, inverse `inv`
- Système linéaire `solve`
- Valeurs/vecteurs propres `eig`
- Normes `norm`, rang `matrix_rank`
- SVD `svd`


In [ ]:

import numpy as np
from numpy.linalg import det, inv, eig, norm, matrix_rank, solve, svd

A = np.array([[3., 1.], [2., 4.]])
b = np.array([7., 10.])

# Opérations de base
trace = np.trace(A)
prod = A @ A
d = det(A)
A_inv = inv(A)
x = solve(A, b)            # solution du système Ax=b

w, V = eig(A)              # valeurs/vecteurs propres
r = matrix_rank(A)
U, S, VT = svd(A)          # SVD

(trace, d, x, w, r, S)



**Exercice 9.1 — Inversibilité & conditionnement**  
1) Écris une fonction `is_invertible(M, tol=1e-12)` qui teste `det(M)` vs 0.  
2) Estime la **sensibilité** d’une résolution `Ax=b` en ajoutant un petit bruit à `A` et en comparant la variation de la solution.

<details>
<summary>✅ Solution (idée)</summary>

```python
from numpy.linalg import det, solve, norm

def is_invertible(M, tol=1e-12):
    return abs(det(M)) > tol

rng = np.random.default_rng(0)
A = np.array([[3.,1.],[2.,4.]])
b = np.array([7.,10.])
x = solve(A,b)

noise = rng.normal(0, 1e-4, size=A.shape)
x_pert = solve(A+noise, b)
relative_change = norm(x_pert - x) / (norm(x) + 1e-12)
is_invertible(A), relative_change
```
</details>



---
## 10) 🧭 Récapitulatif express
- `ndarray` homogène ; maîtriser shape/ndim/dtype/size
- Indexation **et** slicing → **views** vs **copies**
- **Vectorisation** et **broadcasting** : éviter les boucles Python
- Génération aléatoire avec `Generator`
- `np.linalg` pour l’algèbre linéaire



## 11) 📝 Quiz rapide
1) Quelle est la différence entre `ravel()` et `flatten()` ?  
2) Que fait `np.where(cond, A, B)` ?  
3) Explique la règle de **broadcasting** sur `(2,1)` + `(1,3)`.  
4) Quelle est la relation entre `@` et `np.matmul` ?  
5) Pourquoi `np.random.default_rng()` est recommandé par rapport à l’ancienne API ?

<details>
<summary>✅ Corrigé</summary>

1) `ravel()` renvoie une **vue** si possible ; `flatten()` renvoie une **copie**.  
2) Sélectionne élément par élément `A` si `cond` vrai, sinon `B`.  
3) Les dimensions de taille 1 sont étendues : résultat `(2,3)`.  
4) Ce sont deux interfaces du **produit matriciel**.  
5) API moderne, indépendante de l’état global, meilleure reproductibilité.
</details>



## 12) 🎯 Mini-projet — *Pipeline NumPy sans Pandas*
**Tâche** :  
À partir d’une matrice `X` simulée (shape `(1000, 5)`), réaliser :  
1) **Nettoyage** : remplacer les valeurs manquantes (`NaN`) par la moyenne **colonne**.  
2) **Standardisation** : `z-score` colonne par colonne.  
3) **Réduction** : projeter `X` sur 2 composantes via **SVD** (approximation PCA sans scikit).  
4) **Clustering rudimentaire** : assigner chaque point au **centroïde** le plus proche parmi 3 centroïdes initiaux choisis aléatoirement (1 itération).  
5) **Rapport** : afficher les tailles de clusters et la variance expliquée approchée par les 2 premières valeurs singulières.

*(Objectif : réinvestir ufuncs, masques, `nan*`, `linalg` et broadcasting.)*


In [ ]:

import numpy as np

rng = np.random.default_rng(123)
# 1) Données simulées avec quelques NaN
X = rng.normal(0, 1, size=(1000, 5))
mask_nan = rng.random(X.shape) < 0.05
X[mask_nan] = np.nan

# Remplacer NaN par moyenne colonne
col_mean = np.nanmean(X, axis=0, keepdims=True)
inds = np.where(np.isnan(X))
X[inds] = np.take(col_mean, inds[1])

# 2) Standardisation (z-score)
mu = X.mean(axis=0, keepdims=True)
sd = X.std(axis=0, keepdims=True) + 1e-12
Z = (X - mu) / sd

# 3) SVD (approx PCA)
U, S, VT = np.linalg.svd(Z, full_matrices=False)
Z2 = U[:, :2] * S[:2]  # projection en 2D
explained = (S[:2]**2).sum() / (S**2).sum()

# 4) Clustering 1-iter sur 3 centroïdes aléatoires
k = 3
centroids = Z2[rng.choice(len(Z2), size=k, replace=False)]
# distances (N,k)
d2 = ((Z2[:, None, :] - centroids[None, :, :])**2).sum(axis=2)
labels = d2.argmin(axis=1)

# 5) Rapport
sizes = np.bincount(labels, minlength=k)
explained, sizes, centroids



---
## 📚 Ressources
- NumPy docs : https://numpy.org/doc/  
- Random Generator : https://numpy.org/doc/stable/reference/random/  
- Linear algebra : https://numpy.org/doc/stable/reference/routines.linalg.html  

**Prochain notebook** : **Matplotlib — Visualisation 2D/3D** (puis **SciPy**, **Pandas**)
